# Balistique avec et sans frottements

In [ ]:
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#from IPython.display import HTML

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead, Slider

import warnings; warnings.simplefilter('ignore')


output_notebook()

# Equation de la trajectoire d'un projectile avec et sans frottements

On considère un projectile, lancé avec un angle $\alpha$ depuis l'horizontale. Le calcul de la trajectoire  est fait d'abord sans les frottements visqueux de l'air, puis avec.

La trajectoire  **sans les frottements de l'air** est donnée par l'équation paramétrique du temps:

$$\vec r = \Big(
\begin{matrix}t\,v_0\cos\alpha\\ -\frac{1}{2}\,g\,t^2+t\,v_0\sin\alpha
\end{matrix}$$


Le calcul **prenant en compte les frottements de l'air** ($\vec F_F= - b_l \vec v$) est le suivant :


$\sum \vec F = m \vec a = \vec p + \vec F_{F} = -m g \vec e_{y} - b_{l} \vec v$

Projeté sur les vecteurs de base, on obtient: 


$$ \begin{pmatrix} m \ddot{x}\\  m \ddot{y} \end{pmatrix}  =  \begin{pmatrix} 0\\  -mg \end{pmatrix}  + 
\begin{pmatrix} -b_{l} \dot{x}\\  -b_{l} \dot{y} \end{pmatrix} $$

Ce qui donne les équations différentielles suivantes: 


$$\left\{\begin{matrix} m \ddot{x} + b_{l}\dot{x} = 0\\  m \ddot{y} + b_{l}\dot{y} + mg  = 0\end{matrix}\right. $$ 

En définissant $\lambda = \frac{b_{l}}{m}$ on obtient:

$$\left\{\begin{matrix} \ddot{x} + \lambda \dot{x} = 0\\  \ddot{y} + \lambda \dot{y} + g = 0 \end{matrix}\right. $$

 

$$\frac{\mathrm{d} v_{x}}{\mathrm{d} t} + \lambda v_{x} = 0 \rightarrow \frac{\mathrm{d} v_{x}}{\mathrm{d} t} = -  \lambda v_{x} \rightarrow ln \left (  \frac{v_{x}}{v_{0_x}}  \right ) = - \lambda t$$

$$v_x = v_0 + cos \left ( \alpha \right )e^{-\lambda t} \rightarrow x\left ( t \right ) = \frac{v_{0}  cos\left ( \alpha \right )}{\lambda}\left [ 1 - e^{-\lambda t} \right ]$$


La composant $y$ du vecteur position s'obtient comme suit : 

$$\frac{\mathrm{d} v_{y}}{\mathrm{d} t} = - \left ( g + \lambda v_{y} \right ) \rightarrow \frac{\mathrm{d} v_{y}}{g + \lambda v_{y}} = - \mathrm{d} t \rightarrow \frac{ 1}{ - \lambda } ln\left ( \frac{g + \lambda v_{y} }{g + \lambda v_{0_y} } \right ) = - t $$

$$g + \lambda v_{y} = \left (  g + \lambda v_{0} sin \left ( \alpha \right )\right ) e^{-\lambda t} \rightarrow 
v_y = \frac{g + \lambda v_{0} sin \left ( \alpha \right )}{\lambda} e^{-\lambda t} - \frac{g}{\lambda}$$

$$y\left ( t\right ) = - \frac{1}{\lambda } \frac{g + \lambda v_0sin\left ( \alpha \right )}{\lambda } \left [ e^{-\lambda t}  - 1 \right ] - \frac{g t}{\lambda} \rightarrow y\left ( t\right ) = \frac{g + \lambda v_0sin\left ( \alpha \right )}{\lambda^{2} } \left [ 1 - e^{-\lambda t}  \right ] - \frac{g t}{\lambda}$$

Ce qui donne comme équation paramétrique de la trajectoire: 

$$\vec r= \Bigg(
\begin{matrix}\frac{v_{0}  cos\left ( \alpha \right )}{\lambda}\left [ 1 - e^{-\lambda t} \right ]\\
\frac{g + \lambda v_0sin\left ( \alpha \right )}{\lambda^{2} } \left [ 1 - e^{-\lambda t}  \right ] - \frac{g t}{\lambda} 
\end{matrix}$$

Les paramètre du problème sont donc:

-- $v_0$ norme de la vitesse initiale

-- $\alpha$, angle avec l'horizontale

-- $\lambda=b_l/m$ coefficient de frottement 

In [ ]:
g = 9.81 # g: accélération de la pesaneur terrestre
omega = np.pi*25/180 # omega: angle pour les têtes des flèches 

# fonction pour définir les têtes des flèches des vecteurs 
build_arrow_x = lambda v, a, x: np.linspace(v*np.cos(a) - 0.1*v*np.cos(a - x),v*np.cos(a), 10)
build_arrow_y = lambda v, a, x: np.linspace(v*np.sin(a) - 0.1*v*np.sin(a - x),v*np.sin(a), 10)

In [ ]:
# Fonctions auxiliaires pour calculer x et y sans frottements (paramètres: v_0, alpha et t)
traj_sf_x_ = lambda v_0, alpha, t: v_0*np.cos(alpha)*t
traj_sf_y_ = lambda v_0, alpha, t:-0.5*g*t**2+v_0*np.sin(alpha)*t 


# Fonctions auxiliaires pour calculer x et y avec frottements (paramètres: v_0, alpha,lamb et t)
traj_frot_x_ = lambda v_0, alpha, lamb, t: v_0*np.cos(alpha)/lamb * (1 - np.exp(-lamb*t))
traj_frot_y_ = lambda v_0, alpha, lamb, t: (g+lamb*v_0*np.sin(alpha))*(1-np.exp(-lamb*t))/(lamb**2) - t*g/lamb

# Vecteur vitesse initiale
v_init_x_ = lambda v_0, alpha: np.linspace(0, v_0*np.cos(alpha), 2)
v_init_y_ = lambda v_0, alpha: np.linspace(0, v_0*np.sin(alpha), 2)

In [ ]:
## calcul des paramètres

# Définition des paramètres initiaux
v_0_ini = 25 #en m/s
alpha_ini = np.pi*45/180
lamb = 0.5 #lambda

# On divise le temps en 1000 , entre 0 et 15 secondes
t = np.linspace(0,15,1000)

# x et y de t avec frottemets
x = traj_frot_x_(v_0_ini, alpha_ini, lamb, t)
y = traj_frot_y_(v_0_ini, alpha_ini, lamb, t)

#x et y de t sans frottements
x_no_f = traj_sf_x_(v_0_ini, alpha_ini, t)
y_no_f = traj_sf_y_(v_0_ini, alpha_ini, t)

# vecteurs vitesse initiale
v_init_x = v_init_x_(v_0_ini, alpha_ini)
v_init_y = v_init_y_(v_0_ini, alpha_ini)

# Fonctions pour le tracé de la trajectoire avec et sans frottements

In [ ]:

# Définir figure
p = figure(title="Trajectoire du projectile", plot_height=360, plot_width=792, y_range=(0,135), x_range=(0,297), 
           background_fill_color='#ffffff')

# Trajectoire avec frottements 
r = p.line(x, y, color="#8888cc", line_width=4.5, alpha=0.8, legend='Avec frottements')

# Trajectoire sans frottements
r_n =  p.line(x_no_f, y_no_f, color="#b6e740", line_width=4.5, alpha=0.8, legend='Sans frottements')

# Vitesse initiale 
v = p.line(v_init_x, v_init_y, color="#e30d4f", line_width=2, alpha=0.8, legend='Vitesse initiale')
# lignes servant à tracer la pointe de flèche du vecteur vitesse initiale
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini, omega), build_arrow_x(v_0_ini, alpha_ini, omega), color="#e30d4f", line_width=1.5, alpha=0.8, legend='Vitesse initiale')
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini, -omega), build_arrow_x(v_0_ini, alpha_ini, -omega), color="#e30d4f", line_width=1.5, alpha=0.8, legend='Vitesse initiale')

p.xaxis.axis_label = 'x (m)'
p.yaxis.axis_label = 'y (m)'

def update_bullet(v_0, alpha, lamb):
    alpharad =alpha*np.pi/180
    
    r.data_source.data['x'] = traj_frot_x_(v_0, alpharad, lamb, t)
    r.data_source.data['y'] = traj_frot_y_(v_0, alpharad, lamb, t)
    
    r_n.data_source.data['x'] = traj_sf_x_(v_0, alpharad, t)
    r_n.data_source.data['y'] = traj_sf_y_(v_0, alpharad, t)
    
    v.data_source.data['x'] = v_init_x_(v_0, alpharad)
    v.data_source.data['y'] = v_init_y_(v_0, alpharad)
    
    a_up.data_source.data['x'] = build_arrow_x(v_0, alpharad, omega)
    a_up.data_source.data['y'] = build_arrow_y(v_0, alpharad, omega)
    
    a_down.data_source.data['x'] = build_arrow_x(v_0, alpharad, -omega)
    a_down.data_source.data['y'] = build_arrow_y(v_0, alpharad, -omega)
    
    push_notebook()
    


# 1. Tracé proprement dit. 

Utiliser les sliders pour faire varier le coefficient de frottement $\lambda$, la vitesse initiale et l'angle de tir $\alpha$

In [ ]:
show(p, notebook_handle=True)
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$ :'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='alpha :'), \
        lamb=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Lambda :'));

# 2. Étude d'un point de la trajectoire du projectile avec frottements

La figure ci-dessous montre le detail des forces et vitesses en un point de la trajectoire du projectile quand on considére les frottements

In [ ]:
# Définition des fonctions auxiliaires
get_v_x = lambda v, alpha, lamb: v*np.cos(alpha)*np.exp(-lamb*t)
get_v_y = lambda v, alpha, lamb: -g/lamb+(v*np.sin(alpha)+g/lamb)*np.exp(-lamb*t)
get_v   = lambda v, alpha, lamb: np.sqrt((v*np.cos(alpha)*np.exp(-lamb*t))**2+(-g/lamb+(v*np.sin(alpha)+g/lamb)*np.exp(-lamb*t))**2)

# Vitesse au point choisi 
v_p_ = lambda x, y, point, v_x, v_y: [np.linspace(x[point], x[point] + v_x[point], 10), np.linspace(y[point], y[point] + v_y[point], 10)]

# Angle du vecteur vitesse 
ang_v_ = lambda v_x, v_y, point: np.arctan(v_y[point]/v_x[point])

# Force de frottements
f_p_ = lambda x, y, v_x, v_y, lamb, point: [np.linspace(x[point] - lamb*v_x[point], x[point], 10),
                                      np.linspace(y[point] -lamb* v_y[point],y[point], 10)]

# Force de gravité 
fg_ = lambda x, y, point: [np.linspace(x[point], x[point], 10), np.linspace(y[point], y[point]-g, 10)]

In [ ]:
# Définition des paramètres initiaux
v_0_ini = 15
alpha_ini = np.pi*45/180 
lamb = 0.5
point = 200
omega = np.pi*25/180

# Vecteur du temps. Entre 0 et 15s divissé en 1000
t = np.linspace(0,15,1000)

# x et y de t avec frottemets
x = traj_frot_x_(v_0_ini, alpha_ini, lamb, t)
y = traj_frot_y_(v_0_ini, alpha_ini, lamb, t)

# vecteur de vitesse initial
x2 = v_init_x_(v_0_ini, alpha_ini)
y2 = v_init_y_(v_0_ini, alpha_ini)

# vitesse au point choisi
v_x_with = get_v_x(v_0_ini, alpha_ini, lamb)
v_y_with = get_v_y(v_0_ini, alpha_ini, lamb)
v_with = get_v(v_0_ini, alpha_ini, lamb)

# angle vecteur vitesse
angle_v = ang_v_(v_x_with, v_y_with, point)

In [ ]:
# Figure et fonctions pour l'updater 
p = figure(title="Analyse d'un point de la trajectoire du projectile avec frottements", \
           plot_height=450, plot_width=900, y_range=(0,60), x_range=(0,120), background_fill_color='#ffffff', aspect_scale=1)

# trajectoire jusq'au point
r = p.line(x[0:point], y[0:point], color="#8888cc", line_width=4.5, alpha=0.8, legend='Trajectoire avec frottements')
# trajectoire après point
r_end = p.line(x[point:], y[point:], color="#8888cc", line_width=3, alpha=0.6, line_dash='dashed', legend='Trajectoire avec frottements')

# vitesse initiale
v = p.line(x2, y2, color="#e30d4f", line_width=2, alpha=0.2, legend='Vitesse initiale')
# lignes servant à tracer la pointe de flèche du vecteur vitesse initiale
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini, -omega), build_arrow_y(v_0_ini, alpha_ini, omega), color="#e30d4f", line_width=1.5, alpha=0.2, legend='Vitesse initiale')
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini, omega), build_arrow_y(v_0_ini, alpha_ini, -omega), color="#e30d4f", line_width=1.5, alpha=0.2, legend='Vitesse initiale')

# marquer le point dans la trajectoire
mark_point = p.circle(x[point], y[point], size=5)

# vecteur vitesse au point choisi
v_p_x, v_p_y = v_p_(x, y, point, v_x_with, v_y_with)
v_p = p.line(v_p_x, v_p_y, color="#FFB749", line_width=2, legend='Vitesse au point P')

# vecteur de force de frottements
f_p_x, f_p_y = f_p_(x,y,v_x_with, v_y_with, lamb, point)
f_p = p.line(f_p_x, f_p_y, color="#0e11ec", line_width=2, legend="Force de frottement de l'air")

# force de gravité
fg_x, fg_y = fg_(x,y,point)
f_g = p.line(fg_x, fg_y, color="#0eaeec", line_width=2, legend='Poids')


## lignes servant à traces la pointe de flèche des vecteurs

# lignes servant à tracer la pointe de flèche du vecteur vitesse au point choisi
a_down_v = p.line(np.linspace(v_x_with[point]+x[point], v_x_with[point]+x[point] -0.1*v_with[point]*np.cos(-angle_v + np.pi/4 + np.pi*20/180), 10),\
                np.linspace(v_y_with[point]+y[point], v_y_with[point]+y[point] +0.1*v_with[point]*np.sin(angle_v - np.pi/4 - np.pi*1/180), 10),\
                color="#FFB749", line_width=1.5, alpha=0.8, legend='Vitesse au point P')

a_up_v = p.line(np.linspace(v_x_with[point]+x[point], v_x_with[point]+x[point] -0.1*v_with[point]*np.sin(angle_v + np.pi/4 + np.pi*20/180), 10),\
                np.linspace(v_y_with[point]+y[point], v_y_with[point]+y[point] +0.1*v_with[point]*np.cos(angle_v + np.pi/4 + np.pi*20/180), 10),\
                color="#FFB749", line_width=1.5, alpha=0.8, legend='Vitesse au point P')

# lignes servant à tracer la pointe de flèche du vecteur de force de frottements
a_up_f = p.line(np.linspace(x[point] - lamb*v_x_with[point], \
                            x[point] - lamb*v_x_with[point] - 0.1*v_with[point]*np.cos(angle_v - np.pi*(1-160)/180), 10), \
               np.linspace(y[point] - lamb*v_y_with[point], \
                            y[point] - lamb*v_y_with[point] - 0.1*v_with[point]*np.sin(angle_v - np.pi*(1-160)/180), 10), \
                            color='#0e11ec', line_width=1.5, legend="Force de frottement de l'air")

a_down_f = p.line(np.linspace(x[point] - lamb*v_x_with[point], \
                            x[point] - lamb*v_x_with[point] - 0.1*v_with[point]*np.cos(angle_v - np.pi*(1+160)/180), 10), \
               np.linspace(y[point] - lamb*v_y_with[point], \
                            y[point] - lamb*v_y_with[point] - 0.1*v_with[point]*np.sin(angle_v - np.pi*(1+160)/180), 10), \
                            color='#0e11ec', line_width=1.5,legend="Force de frottement de l'air")

# lignes servant à tracer la pointe de flèche du vecteur de force de gravité
a_up_g = p.line(np.linspace(x[point], x[point] + 0.3, 10), \
               np.linspace(0.7*y[point], 0.8*y[point], 10), color="#0eaeec", line_width=1.5, legend='Poids')
a_down_g = p.line(np.linspace(x[point], x[point] - 0.3, 10), \
               np.linspace(0.7*y[point], 0.8*y[point], 10), color="#0eaeec", line_width=1.5, legend='Poids')



p.xaxis.axis_label = 'x (m)'
p.yaxis.axis_label = 'y (m)'
p.legend.click_policy="hide"

def update_bullet(point, v_0, alpha, lamb):
    alpharad = alpha*np.pi/180
    
    
    x = traj_frot_x_(v_0, alpharad, lamb, t)
    y = traj_frot_y_(v_0, alpharad, lamb, t)
    
    v_x_with = get_v_x(v_0, alpharad, lamb)
    v_y_with = get_v_y(v_0, alpharad, lamb)
    v_with = get_v(v_0, alpharad, lamb)

    # angle vecteur vitesse
    angle_v = ang_v_(v_x_with, v_y_with, point)

    # Mettre à jour la trajectoire
    r.data_source.data['x'] = x[0:point]
    r.data_source.data['y'] = y[0:point]
    r_end.data_source.data['x'] = x[point:]
    r_end.data_source.data['y'] = y[point:]
    
    
    # Mettre à jour vitesse initiale
    v_init_x = v_init_x_(v_0, alpharad)
    v_init_y = v_init_y_(v_0, alpharad)
    v.data_source.data['x'] = v_init_x
    v.data_source.data['y'] = v_init_y
    
    a_up.data_source.data['x'] = v_init_x[-1] + np.linspace(0, -np.cos(alpharad + omega), 10)
    a_up.data_source.data['y'] = v_init_y[-1] + np.linspace(0, -np.sin(alpharad + omega), 10)
    a_down.data_source.data['x'] = v_init_x[-1] + np.linspace(0, -np.cos(alpharad - omega), 10)
    a_down.data_source.data['y'] = v_init_y[-1] + np.linspace(0, -np.sin(alpharad - omega), 10)

    
    # Mettre à jour vitesse au point choisi
    v_p_x, v_p_y = v_p_(x, y, point, v_x_with, v_y_with)
    v_p.data_source.data['x'] = v_p_x
    v_p.data_source.data['y'] = v_p_y

    a_down_v.data_source.data['x'] = v_x_with[point]+x[point] + np.linspace(0, -np.cos(angle_v + omega), 10)
    a_down_v.data_source.data['y'] = v_y_with[point]+y[point] + np.linspace(0, -np.sin(angle_v + omega), 10)
    a_up_v.data_source.data['x'] = v_x_with[point]+x[point] + np.linspace(0, -np.cos(angle_v - omega), 10)
    a_up_v.data_source.data['y'] = v_y_with[point]+y[point] + np.linspace(0, -np.sin(angle_v - omega), 10)

    # Mettre à jour frottements
    f_p_x, f_p_y = f_p_(x,y,v_x_with, v_y_with, lamb, point)
    f_p.data_source.data['x'] = f_p_x
    f_p.data_source.data['y'] = f_p_y
    
    a_up_f.data_source.data['x'] = x[point] - lamb*v_x_with[point] + np.linspace(0, +np.cos(angle_v + omega), 10)
    a_up_f.data_source.data['y'] = y[point] - lamb*v_y_with[point] + np.linspace(0, +np.sin(angle_v + omega), 10)
    a_down_f.data_source.data['x'] = x[point] - lamb*v_x_with[point] + np.linspace(0, +np.cos(angle_v - omega), 10)
    a_down_f.data_source.data['y'] = y[point] - lamb*v_y_with[point] + np.linspace(0, +np.sin(angle_v - omega), 10)
    
    # Mettre à jour gravité
    fg_x, fg_y = fg_(x,y,point)
    f_g.data_source.data['x'] = fg_x
    f_g.data_source.data['y'] = fg_y
    a_up_g.data_source.data['x'] = np.linspace(x[point], x[point] + 0.3, 10)
    a_up_g.data_source.data['y'] = np.linspace(y[point]-g, (y[point]-g)+1, 10)
    a_down_g.data_source.data['x'] = np.linspace(x[point], x[point] - 0.3, 10)
    a_down_g.data_source.data['y'] = np.linspace(y[point]-g, (y[point]-g)+1, 10)
    
    push_notebook()
    
                                               
show(p, notebook_handle=True)
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=40, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'), \
        lamb=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Lambda :'), \
        point=widgets.IntSlider(min=0,max=300,step=5,value=150, description='Point temporelle :'));


# 3 Trajectoire d'un projectile lancé sur un terrain en pente

In [ ]:
# Fonctions auxiliaires
slope_ = lambda x, theta: x*np.tan(theta) 

# get_max_point
#get_max_ = lambda y, slope_y: np.where(y/slope_y < 1)[0][0]

In [ ]:
# Définition des paramètres initiaux
v_0 = 25
alpharad = np.pi*45/180
thetarad = np.pi*5/180
lamb = 0.5

# Vecteur du temps. Entre 0 et 15s, divissé en 1000
t = np.linspace(0,15,1000)

# trajectoires avec frottemets
trajectory_x_with = traj_frot_x_(v_0, alpharad+thetarad, lamb, t)
trajectory_y_with = traj_frot_y_(v_0, alpharad+thetarad, lamb, t)

# trajectoires sans frottemets
trajectory_x_without = traj_sf_x_(v_0, alpharad+thetarad, t)
trajectory_y_without = traj_sf_y_(v_0, alpharad+thetarad, t)

# vecteurs vitesse initiale
v_init_x = v_init_x_(v_0_ini, alpha_ini)
v_init_y = v_init_y_(v_0_ini, alpha_ini)

# pente
slope_height = slope_(trajectory_x_without, thetarad)

# vecteur de vitesse initial
x2 = v_init_x_(v_0, alpharad+thetarad)
y2 = v_init_y_(v_0, alpharad+thetarad)

In [ ]:
# Figure
p = figure(title="Trajectoire d'un projectile lancé sur un terrain en pente", plot_height=432, plot_width=950, y_range=(0,135), x_range=(0,297), \
        background_fill_color='#ffffff')

# trajectoire avec frottements
r = p.line(trajectory_x_with, trajectory_y_with, color="#8888cc", \
           line_width=4.5, alpha=0.8, legend='Avec frottements')

# trajectoire sans frottements
r_n =  p.line(trajectory_x_without, trajectory_y_without, color="#b6e740", \
              line_width=4.5, alpha=0.8, legend='Sans frottements')

# vitesse initiale
v = p.line(x2, y2, color="#e30d4f", line_width=2, alpha=0.8, legend='Vitesse initiale')

# lignes servant à tracer la pointe de flèche du vecteur de vitesse initiale
a_up = p.line(build_arrow_x(v_0_ini, alpha_ini+thetarad, omega), build_arrow_y(v_0_ini, alpha_ini+thetarad, omega), color="#e30d4f", line_width=1.5, alpha=0.8)
a_down = p.line(build_arrow_x(v_0_ini, alpha_ini+thetarad, -omega), build_arrow_y(v_0_ini, alpha_ini+thetarad, -omega), color="#e30d4f", line_width=1.5, alpha=0.8)

# pente
s = p.line(np.linspace(0,500,1000), slope_height, color='firebrick', line_width=3, alpha=0.3)

# point d'atterrissage
x_at_sans = p.circle(x=[0], y=[0], size=8, line_color="#32CD32", fill_color="#32CD32", legend='Point atterrissage sans frottements')
p_at_sans = p.circle(x=[0], y=[0], size=8, line_color="#32CD32", fill_color="#32CD32", legend='Point atterrissage sans frottements')

x_at_avec = p.circle(x=[0], y=[0], size=8, line_color="#3288bd", fill_color="#3288bd", legend='Point atterrissage avec frottements')
p_at_avec = p.circle(x=[0], y=[0], size=8, line_color="#3288bd", fill_color="#3288bd", legend='Point atterrissage avec frottements')



p.xaxis.axis_label = 'y (m)'
p.yaxis.axis_label = 'x (m)'

def update_bullet(v_0, alpha, lamb, theta):
    alpharad=alpha*np.pi/180
    thetarad = theta*np.pi/180
    
    tan_theta = np.tan(thetarad)
    x_sans = traj_sf_x_(v_0, alpharad+thetarad, t)
    y_sans = traj_sf_y_(v_0, alpharad+thetarad, t)

    slope_height = slope_(x_sans, thetarad)

    idx = np.where(np.round(y_sans/x_sans,2) <= round(tan_theta,2))[0][0]
    
    r_n.data_source.data['x'] = x_sans[:idx]
    r_n.data_source.data['y'] = y_sans[:idx]
    
    x_avec = traj_frot_x_(v_0, alpharad+thetarad, lamb, t)
    y_avec = traj_frot_y_(v_0, alpharad+thetarad, lamb, t)
    idx_avec = np.where(np.round(y_avec/x_avec,2) <= round(tan_theta,2))[0][0]
    
    r.data_source.data['x'] = x_avec[:idx_avec+1]
    r.data_source.data['y'] = y_avec[:idx_avec+1]
    
    v.data_source.data['x'] = v_init_x_(v_0, alpharad+thetarad)
    v.data_source.data['y'] = v_init_y_(v_0, alpharad+thetarad)
    
    a_up.data_source.data['x'] = build_arrow_x(v_0, alpharad+thetarad, omega)
    a_up.data_source.data['y'] = build_arrow_y(v_0, alpharad+thetarad, omega)
    
    a_down.data_source.data['x'] = build_arrow_x(v_0, alpharad+thetarad, -omega)
    a_down.data_source.data['y'] = build_arrow_y(v_0, alpharad+thetarad, -omega)

    #s.data_source.data['x'] = x_sans
    s.data_source.data['y'] = slope_(np.linspace(0,500,1000), thetarad)
    
    x_at_sans.data_source.data['x'] = [x_sans[idx]]
    p_at_sans.data_source.data['x'] = [x_sans[idx]]
    p_at_sans.data_source.data['y'] = [y_sans[idx]]
    
    x_at_avec.data_source.data['x'] = [x_avec[idx_avec+1]]
    p_at_avec.data_source.data['x'] = [x_avec[idx_avec+1]]
    p_at_avec.data_source.data['y'] = [y_avec[idx_avec+1]]
    
    
    if (alpha+theta)>90:
        print("Attention! L'angle de lancement par rapport à l'horizontale dépasse les 90º")
    
    push_notebook()
    
show(p, notebook_handle=True);
interact(update_bullet, \
         v_0=widgets.IntSlider(min=0,max=50,step=1,value=25, description='$v_0$:'), \
         alpha=widgets.IntSlider(min=0,max=90,step=1,value=45, description='Alpha :'), \
        lamb=widgets.FloatSlider(min=0.00001,max=1,step=0.001,value=0.5, description='Lambda :'),\
         theta=widgets.FloatSlider(min=0,max=45,step=1,value=5, description='Theta :'));